In [2]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from IPython import get_ipython

from ES import sepCEM, Control

# %% [markdown]
# This notebook is a little easier for beginners because it uses pytorch. You need to clone a repo to get it working:
# 
# ```sh
# # you need this repo, so clone it
# git clone https://github.com/wassname/DeepRL.git
# cd DeepRL
# git reset --hard aeae2c5d585e5853dc638968b1f090eb60abd351
# cd ..
# mkdir data log evaluation_log
# ```
# 
# This contains some minor modifications from https://github.com/ShangtongZhang/DeepRL.git
# 
# The notebook tries DPPG with the [EIIE model](https://arxiv.org/pdf/1706.10059.pdf)

# %%


# %% [markdown]
# I also uncommented reward normalization in DDPG_agent.py#L64 because otherwise my small reward les to large Q's, inf losses, and NaN actions and weights.

# %%
# plotting
get_ipython().run_line_magic('matplotlib', 'notebook')
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# numeric
import numpy as np
from numpy import random
import pandas as pd

# utils
from tqdm import tqdm_notebook as tqdm
from collections import Counter
import tempfile
import logging
import time
import datetime
from copy import deepcopy
from util import to_numpy


# logging
logger = log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
logging.basicConfig()
log.info('%s logger started.', __name__)


# %%
import os
os.sys.path.append(os.path.abspath('.'))
os.sys.path.append(os.path.abspath('DeepRL'))
get_ipython().run_line_magic('reload_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')


# %%



# %%
# params
window_length = 50
steps = 128


# %%
# save dir
import datetime
ts = datetime.datetime.utcnow().strftime('%Y%m%d_%H-%M-%S')

save_path = './outputs/pytorch-DDPG/pytorch-DDPG-EIIE-action-crypto-%s.model' % ts
print(save_path)
try:
    os.makedirs(os.path.dirname(save_path))
except OSError:
    pass


# %%
# setup tensorboard logging
from tensorboard_logger import configure, log_value
tag = 'ddpg-' + ts
print('tensorboard --logdir '+"runs/" + tag)
try:
    configure("runs/" + tag)
except ValueError as e:
    print(e)
    pass


# %%


# %% [markdown]
# # Env

# %%
from rl_portfolio_management.environments.portfolio import PortfolioEnv
from rl_portfolio_management.util import MDD, sharpe, softmax
from rl_portfolio_management.wrappers import SoftmaxActions, TransposeHistory, ConcatStates

df_train = pd.read_hdf('./data/poloniex_30m.hf',key='train')
df_test = pd.read_hdf('./data/poloniex_30m.hf',key='test')


# %%



# %%
import gym
class DeepRLWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.render_on_reset = False
        
        self.state_dim = self.observation_space.shape
        self.action_dim = self.action_space.shape[0]
        
        self.name = 'PortfolioEnv'
        self.success_threshold = 2
        
    def normalize_state(self, state):
        return state
    
    def step(self, action):
        state, reward, done, info =self.env.step(action)
        reward*=1e4 # often reward scaling is important sooo...
        return state, reward, done, info
    
    def reset(self):        
        # here's a roundabout way to get it to plot on reset
        if self.render_on_reset: 
            self.env.render('notebook')

        return self.env.reset()


# %%
def task_fn():
    env = PortfolioEnv(df=df_train, steps=steps, output_mode='EIIE')
    env = TransposeHistory(env)
    env = ConcatStates(env)
    env = SoftmaxActions(env)
    env = DeepRLWrapper(env)
    return env

def task_fn_test():
    env = PortfolioEnv(df=df_test, steps=steps, output_mode='EIIE')
    env = TransposeHistory(env)
    env = ConcatStates(env)
    env = SoftmaxActions(env)
    env = DeepRLWrapper(env)
    return env
    
# sanity check
task = task_fn()
task.reset().shape, task.step(task.action_space.sample())[0].shape

# %% [markdown]
# # Agent and models

# %%
# load
import pickle
import shutil

def save_ddpg(agent):
    agent_type = agent.__class__.__name__
    save_file = 'data/%s-%s-model-%s.bin' % (agent_type, config.tag, agent.task.name)
    agent.save(save_file)
    print(save_file)
    

def load_ddpg(agent):
    agent_type = agent.__class__.__name__
    save_file = 'data/%s-%s-model-%s.bin' % (agent_type, config.tag, agent.task.name)
    new_states = pickle.load(open(save_file, 'rb'))
    states = agent.worker_network.load_state_dict(new_states)


def load_stats_ddpg(agent):
    agent_type = agent.__class__.__name__
    online_stats_file = 'data/%s-%s-online-stats-%s.bin' % (
                    agent_type, config.tag, agent.task.name)
    try:
        steps, rewards = pickle.load(open(online_stats_file, 'rb'))
    except FileNotFoundError:
        steps =[]
        rewards=[]
    df_online = pd.DataFrame(np.array([steps, rewards]).T, columns=['steps','rewards'])
    if len(df_online):
        df_online['step'] = df_online['steps'].cumsum()
        df_online.index.name = 'episodes'
    
    stats_file = 'data/%s-%s-all-stats-%s.bin' % (agent_type, config.tag, agent.task.name)

    try:
        stats = pickle.load(open(stats_file, 'rb'))
    except FileNotFoundError:
        stats = {}
    df = pd.DataFrame(stats["test_rewards"], columns=['rewards'])
    if len(df):
#         df["steps"]=range(len(df))*50

        df.index.name = 'episodes'
    return df_online, df


# %%



# %%
import logging
from DeepRL.agent import ProximalPolicyOptimization
from DeepRL.network import DisjointActorCriticNet #, DeterministicActorNet, DeterministicCriticNet
from DeepRL.component import GaussianPolicy, HighDimActionReplay, OrnsteinUhlenbeckProcess
from DeepRL.utils import Config, Logger
import gym
import torch
gym.logger.setLevel(logging.INFO)

# %% [markdown]
# # Alg

# %%
# Modified from https://github.com/ShangtongZhang/DeepRL to log to tensorboard

from DeepRL.utils.normalizer import Normalizer

null_normaliser = lambda x:x

# USE_CUDA = torch.cuda.is_available()
# if USE_CUDA:
#     print("using CUDA")
#     FloatTensor = torch.cuda.FloatTensor
# else:
#     print("using CPU")
#     FloatTensor = torch.FloatTensor



def evaluate(actor, env, max_steps, memory=None, n_episodes=1, random=False, noise=None, deterministic=False):
    """
    Computes the score of an actor on a given number of runs,
    fills the memory if needed
    """

    if not random:
        def policy(state):
            
            state = torch.FloatTensor(np.array([state])) #.reshape(-1))
            #print("Action Shape: ", np.shape(state))
            action = actor(state).cpu().data.numpy().flatten()

            if noise is not None:
                action += noise.sample()

            max_action = int(env.action_space.high[0])
            return np.clip(action, -max_action, max_action)

    else:
        def policy(state):
            return env.action_space.sample()

    scores = []
    steps = 0

    for _ in range(n_episodes):

        score = 0
        obs = deepcopy(env.reset())
        #  experiences = self.replay.sample()
        # states, actions, rewards, next_states, terminals = experiences
        done = False

        obs_l = []
        rewards = []
        dones = []
        s = []

        while not done:

            # get next action and act
            action = policy(obs)
            n_obs, reward, done, _ = env.step(action)
            done_bool = 0 if steps + 1 == max_steps else float(done)
            score += reward
            steps += 1

            if deterministic:
                obs_l.append(n_obs)
                rewards.append(reward)
                dones.append(done)
                s.append(steps)
                

                # filewriter = open("test_fitness.csv", "a")
                # filewriter.write("Step,Average Fitness")
                # filewriter.write(str(self.total_steps) + "," + 
                #                 str(np.mean(fitness)) + "," +
                #                 )
                # filewriter.close()
            # adding in memory
            if memory is not None:
                memory.feed((obs, action, reward, n_obs, done_bool))
            obs = n_obs

            # # render if needed
            # if render:
            #     env.render()

            # reset when done
            if done:
                env.reset()

        if deterministic:
            df = pd.DataFrame({
                'steps':s,
                # 'observation':obs_l,
                'reward':rewards,
                'done':dones
            })
            df.to_csv('ddpg_cem_test_results.csv')

        scores.append(score)

    return np.mean(scores), steps





class DDPGAgent:
    def __init__(self, config):
        self.config = config
        self.task = config.task_fn()
        self.worker_network = config.network_fn()
        self.target_network = config.network_fn()
        self.target_network.load_state_dict(self.worker_network.state_dict())
        self.actor_opt = config.actor_optimizer_fn(self.worker_network.actor.parameters())
        self.critic_opt = config.critic_optimizer_fn(self.worker_network.critic.parameters())
        self.replay = config.replay_fn()
        self.random_process = config.random_process_fn()
        self.criterion = nn.MSELoss()
        self.total_steps = 0
        self.sigma_init = 1e-3
        self.damp = 1e-3
        self.damp_limit = 1e-5
        self.pop_size = 10
        self.elitism = 'elitism'
        self.n_grad = 5
        self.start_steps = 1000 #10000
        self.n_episodes = 1
        self.n_noisy = 0

        self.state_normalizer = Normalizer(self.task.state_dim) # null_normaliser # 
        self.reward_normalizer = Normalizer(1)

        self.es = sepCEM(self.worker_network.actor.get_size(), mu_init=self.worker_network.actor.get_params(), 
            sigma_init=self.sigma_init, damp=self.damp, damp_limit=self.damp_limit,
            pop_size=self.pop_size, antithetic=not self.pop_size % 2, parents=self.pop_size // 2,
            elitism=self.elitism)

    def soft_update(self, target, src):
        for target_param, param in zip(target.parameters(), src.parameters()):
            target_param.data.copy_(target_param.data * (1.0 - self.config.target_network_mix) +
                                    param.data * self.config.target_network_mix)

    def save(self, file_name):
        with open(file_name, 'wb') as f:
            torch.save(self.worker_network.state_dict(), f)

    def episode(self, deterministic=False, video_recorder=None):
        self.random_process.reset_states()
        state = self.task.reset()
        state = self.state_normalizer(state)

        config = self.config
        actor = self.worker_network.actor
        actor_t = self.worker_network.actor
        critic = self.worker_network.critic
        critic_t = self.worker_network.critic
        # target_actor = self.target_network.actor
        # target_critic = self.target_network.critic

        # Initialize our fitness and evolutionary params
       
        # fitness_ = []

        step_cpt = 0
        actor_steps = 0
        #total_reward = 0.0
        while True:

            fitness = []
            es_params = self.es.ask(self.pop_size)
            actor_steps = 0


            if not deterministic:
                # udpate the rl actors and the critic
                if self.total_steps > self.start_steps:

                    for i in range(self.n_grad):

                        # set params
                        actor.set_params(es_params[i])
                        actor_t.set_params(es_params[i])
                        actor.optimizer = self.actor_opt

                        # critic update
                        for _ in range(actor_steps // self.n_grad):
                            critic.update(self.replay, actor, critic_t)

                        # actor update
                        for _ in range(actor_steps):
                            actor.update(self.replay, critic, actor_t)

                        # get the params back in the population
                        es_params[i] = actor.get_params()   

                # evaluate noisy actor(s)
                for i in range(self.n_noisy):
                    actor.set_params(es_params[i])
                    f, steps = evaluate(actor, self.task, self.config.max_episode_length, 
                                        memory=self.replay, n_episodes=self.n_episodes,
                                         noise=self.random_process, deterministic=deterministic)
                    actor_steps += steps
                    #print('Noisy actor {} fitness:{}'.format(i, f))

            # evaluate all actors
            for params in es_params:

                actor.set_params(params)
                f, steps = evaluate(actor, self.task, self.config.max_episode_length, 
                                    memory=self.replay, n_episodes=self.n_episodes, 
                                    noise=None, deterministic=deterministic)
                actor_steps += steps
                fitness.append(f)

                # print scores
                #print('Actor fitness: {}'.format(f))

            # update es
            if not deterministic:
                self.es.tell(es_params, fitness)

            # update step counts
            # self.total_steps += actor_steps
            step_cpt += actor_steps


            # actor.eval()
            # action = actor.predict(np.stack([state])).flatten()
            # if not deterministic:
            #     action += self.random_process.sample()
            # next_state, reward, done, info = self.task.step(action)
            # if video_recorder is not None:
            #     video_recorder.capture_frame()
            # print("Done before done:", done)
            done = config.max_episode_length and self.total_steps >= config.max_episode_length
            # next_state = self.state_normalizer(next_state) * config.reward_scaling
            # total_reward += reward
            
#             # tensorboard logging
#             prefix = 'test_' if deterministic else ''
#             log_value(prefix + 'reward', reward, self.total_steps)
# #             log_value(prefix + 'action', action, steps)
#             log_value('memory_size', self.replay.size(), self.total_steps)     
#             for key in info:
#                 log_value(key, info[key], self.total_steps)     
            
            # reward = self.reward_normalizer(reward)

            # if not deterministic:
            #     self.replay.feed([state, action, reward, next_state, int(done)])
            #     self.total_steps += 1

            if self.total_steps % 10 == 0:
                print("Total Steps:", self.total_steps, " Average fitness:", np.mean(fitness))
            self.total_steps += 1
            # state = next_state

            if done or deterministic:
                # print("max_ep_length:", config.max_epsiode_length)
                # print("total_steps:", self.total_steps)
                break
        return np.mean(fitness), step_cpt 


            # TODO Check what we might need from this
            # if not deterministic and self.replay.size() >= config.min_memory_size:
            #     self.worker_network.train()
            #     experiences = self.replay.sample()
            #     states, actions, rewards, next_states, terminals = experiences
            #     q_next = target_critic.predict(next_states, target_actor.predict(next_states))
            #     terminals = critic.to_torch_variable(terminals).unsqueeze(1)
            #     rewards = critic.to_torch_variable(rewards).unsqueeze(1)
            #     q_next = config.discount * q_next * (1 - terminals)
            #     q_next.add_(rewards)
            #     q_next = q_next.detach()
            #     q = critic.predict(states, actions)
            #     critic_loss = self.criterion(q, q_next)

                #
                # critic.zero_grad()
                # self.critic_opt.zero_grad()
                # critic_loss.backward()
                # if config.gradient_clip:
                #     grad_critic = nn.utils.clip_grad_norm(self.worker_network.parameters(), config.gradient_clip)
                # self.critic_opt.step()

                # actions = actor.predict(states, False)
                # var_actions = Variable(actions.data, requires_grad=True)
                # q = critic.predict(states, var_actions)
                # q.backward(torch.ones(q.size()))

                # actor.zero_grad()
                # self.actor_opt.zero_grad()
                # actions.backward(-var_actions.grad.data)
                # if config.gradient_clip:
                #     grad_actor = nn.utils.clip_grad_norm(self.worker_network.parameters(), config.gradient_clip)
                # self.actor_opt.step()
                
                # tensorboard logging
                # log_value('critic_loss', critic_loss.cpu().data.numpy().squeeze(), self.total_steps)
                # log_value('loss_action', -q.sum(), self.total_steps)
                # if config.gradient_clip:
                #     log_value('grad_critic', grad_critic, self.total_steps)
                #     log_value('grad_actor', grad_actor, self.total_steps)

                # self.soft_update(self.target_network, self.worker_network)

         #total_reward


# %%


# %% [markdown]
# # Model

# %%
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


# %%
task.state_dim, task.action_dim


# %%

from DeepRL.network.base_network import BasicNet

class DeterministicActorNet(nn.Module, BasicNet):
    def __init__(self,
                 state_dim,
                 action_dim,
                 action_gate,
                 action_scale,
                 gpu=False,
                 batch_norm=False,
                 non_linear=F.relu):
        super(DeterministicActorNet, self).__init__()

        stride_time = state_dim[1] - 1 - 2 #
        features = task.state_dim[0]
        h0 = 2
        h1 = 30
        self.conv1 = nn.Conv2d(features, h0, (3, 1))
        self.conv2 = nn.Conv2d(h0, h1, (stride_time, 1), stride=(stride_time, 1))
        self.conv3 = nn.Conv2d((h1+1), 1, (1, 1))

        self.action_scale = action_scale
        self.action_gate = action_gate
        self.non_linear = non_linear

        if batch_norm:
            self.bn1 = nn.BatchNorm1d(h0)
            self.bn2 = nn.BatchNorm1d(h1)

        self.batch_norm = batch_norm
        BasicNet.__init__(self, None, gpu, False)

        self.tau = 0.005

    def forward(self, x):
        x = self.to_torch_variable(x)
        # print("shape of x: ", np.shape(x))
        w0 = x[:,:1,:1,:] # weights from last step 
        x = x[:,:,1:,:]
        
        phi0 = self.non_linear(self.conv1(x))
        if self.batch_norm:
            phi0 = self.bn1(phi0)
        phi1 = self.non_linear(self.conv2(phi0))
        h = torch.cat([phi1,w0], 1)
        if self.batch_norm:
            h = self.bn2(h)
        
        action = self.conv3(h)
        
        # add cash_bias before we softmax
        cash_bias_int = 0
        cash_bias = self.to_torch_variable(torch.ones(action.size())[:,:,:,:1] * cash_bias_int)
        action = torch.cat([cash_bias, action], -1)
        
        batch_size = action.size()[0]
        action = action.view((batch_size,-1))
        if self.action_gate:
            action = self.action_scale * self.action_gate(action)
        return action

    def update(self, memory, critic, actor_t):

            # Sample replay buffer
            states, _, _, _, _ = memory.sample()

            # Compute actor loss
            actor_loss = -critic(states, self(states)).mean()

            # Optimize the actor
            self.optimizer.zero_grad()
            actor_loss.backward()
            self.optimizer.step()

            # Update the frozen target models
            for param, target_param in zip(self.parameters(), actor_t.parameters()):
                target_param.data.copy_(
                    self.tau * param.data + (1 - self.tau) * target_param.data)

    def predict(self, x, to_numpy=True):
        y = self.forward(x)
        if to_numpy:
            y = y.cpu().data.numpy()
        return y

    def set_params(self, params):
        """
        Set the params of the network to the given parameters
        """
        cpt = 0
        for param in self.parameters():
            tmp = np.product(param.size())

            if torch.cuda.is_available():
                param.data.copy_(torch.from_numpy(
                    params[cpt:cpt + tmp]).view(param.size()).cuda())
            else:
                param.data.copy_(torch.from_numpy(
                    params[cpt:cpt + tmp]).view(param.size()))
            cpt += tmp

    def get_params(self):
        """
        Returns parameters of the actor
        """
        return deepcopy(np.hstack([to_numpy(v).flatten() for v in
                                   self.parameters()]))

    def get_grads(self):
        """
        Returns the current gradient
        """
        return deepcopy(np.hstack([to_numpy(v.grad).flatten() for v in self.parameters()]))

    def get_size(self):
        """
        Returns the number of parameters of the network
        """
        return self.get_params().shape[0]


class DeterministicCriticNet(nn.Module, BasicNet):
    def __init__(self,
                 state_dim,
                 action_dim,
                 gpu=False,
                 batch_norm=False,
                 non_linear=F.relu,
                 discount=0.99):
        super(DeterministicCriticNet, self).__init__()
        stride_time = state_dim[1] - 1 - 2 #
        self.features = features = task.state_dim[0]
        h0=2
        h1=20
        self.action = actions = action_dim -1
        self.conv1 = nn.Conv2d(features, h0, (3, 1))
        self.conv2 = nn.Conv2d(h0, h1, (stride_time, 1), stride=(stride_time, 1))
        self.layer3 = nn.Linear((h1+2)*actions, 1)
        self.non_linear = non_linear
        self.discount = discount
        self.tau = 0.005
        #self.config = config
        self.optimizer = config.critic_optimizer_fn(self.parameters())

        if batch_norm:
            self.bn1 = nn.BatchNorm1d(h0)
            self.bn2 = nn.BatchNorm1d(h1)
        self.batch_norm = batch_norm

        BasicNet.__init__(self, None, gpu, False)


    def forward(self, x, action):
        x = self.to_torch_variable(x)
        action = self.to_torch_variable(action)[:,None,None,:-1] # remove cash bias
        
        w0 = x[:,:1,:1,:] # weights from last step 
        x = x[:,:,1:,:]
        
        phi0 = self.non_linear(self.conv1(x))
        if self.batch_norm:
            phi0 = self.bn1(phi0)
        phi1 = self.non_linear(self.conv2(phi0))
        h = torch.cat([phi1,w0,action], 1)
        if self.batch_norm:
            h = self.bn2(h)
        
        batch_size = x.size()[0]
        action = self.layer3(h.view((batch_size,-1)))
        return action

    def update(self, memory, actor_t, critic_t):

            # Sample replay buffer
            states, actions, rewards, n_states, dones = memory.sample()

            # Q target = reward + discount * Q(next_state, pi(next_state))
            with torch.no_grad():
                target_Q = critic_t(n_states, actor_t(n_states))
                # print("original shape target_Q: ", np.shape(target_Q))
                target_Q = target_Q * (1 - np.array([dones],dtype=np.float).reshape([64,1])) * np.array([self.discount],dtype=np.float) + np.array([rewards],dtype=np.float).reshape([64,1])
                # print(target_Q.type())
            # print("dones: ", type(dones))
            # print("n_states:", type(n_states))
            # print("actions: ", type(actions))
            # print("states: ", type(states))
            # print("rewards: ", type(rewards))
            # print("target_Q: ", type(target_Q))
            # print()
            # print("shape dones: ", np.shape(dones))
            # print("shape n_states:", np.shape(n_states))
            # print("shape actions: ", np.shape(actions))
            # print("shape states: ", np.shape(states))
            # print("shape rewards: ", np.shape(rewards))
            # print("shape target_Q: ", np.shape(target_Q))
            # print()
            # Get current Q estimates
            current_Q = self(states, actions)
            # print(current_Q.type())

            # print("currentQ: ", current_Q)



            # Compute critic loss
            critic_loss = nn.MSELoss()(current_Q, target_Q.float())

            # Optimize the critic
            self.optimizer.zero_grad()
            critic_loss.backward()
            self.optimizer.step()

            # Update the frozen target models
            for param, target_param in zip(self.parameters(), critic_t.parameters()):
                target_param.data.copy_(
                    self.tau * param.data + (1 - self.tau) * target_param.data)


    def predict(self, x, action):
        return self.forward(x, action)


# %%


# %% [markdown]
# # Config

# %%



# %%
config = Config()
config.task_fn = task_fn
task = config.task_fn()
config.actor_network_fn = lambda: DeterministicActorNet(
    task.state_dim, task.action_dim, action_gate=None, action_scale=1.0, non_linear=F.relu, batch_norm=False, gpu=False)
config.critic_network_fn = lambda: DeterministicCriticNet(
    task.state_dim, task.action_dim, non_linear=F.relu, batch_norm=False, gpu=False)
config.network_fn = lambda: DisjointActorCriticNet(config.actor_network_fn, config.critic_network_fn)
config.actor_optimizer_fn = lambda params: torch.optim.Adam(params, lr=4e-5)
config.critic_optimizer_fn =    lambda params: torch.optim.Adam(params, lr=5e-4, weight_decay=0.001)
config.replay_fn = lambda: HighDimActionReplay(memory_size=600, batch_size=64)
config.random_process_fn =     lambda: OrnsteinUhlenbeckProcess(size=task.action_dim, theta=0.15, sigma=0.2, sigma_min=0.00002, n_steps_annealing=10000)
config.discount = 0.0

config.min_memory_size = 50
config.target_network_mix = 0.001
config.max_episode_length = 500  
config.target_network_mix = 0.01
config.noise_decay_interval = 100000
config.gradient_clip = 20
config.min_epsilon = 0.1


# sigma_init = 1e-3
# damp = 1e-3
# damp_limit = 1e-5
# pop_size = 10
# elitism = 'elitism'


# Many papers have found rewards scaling to be an important parameter. But while they focus on the scaling factor
# I think they should focus on the end variance with a range of 200-400. e.g. https://arxiv.org/pdf/1709.06560.pdf
# Hard to tell for sure without experiments to prove it
config.reward_scaling = 1000

# config.test_interval = 10 # ORIGINALLY
config.test_interval = 2 # TODO: Remove (quick test)
config.test_repetitions = 1
# config.save_interval = 40 # ORIGINALLY
config.save_interval = 4 # TODO: Remove (quick test)
config.logger = Logger('./log', gym.logger)
config.tag = tag


agent = DDPGAgent(config)
agent

# es = sepCEM(agent.worker_network.actor.get_size(), mu_init=agent.worker_network.actor.get_params(), sigma_init=sigma_init, damp=damp, damp_limit=damp_limit,
#             pop_size=pop_size, antithetic=not pop_size % 2, parents=pop_size // 2, elitism=elitism)



# %%


# %% [markdown]
# # Train


INFO:__main__:__main__ logger started.
[2020-05-01 18:22:59,516] __main__ logger started.
C:\Github\Senior_Sem\Ryan_ddpg-stock-trading-code\rl_portfolio_management\environments\portfolio.py:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = df.as_matrix().reshape(


./outputs/pytorch-DDPG/pytorch-DDPG-EIIE-action-crypto-20200501_22-22-59.model
tensorboard --logdir runs/ddpg-20200501_22-22-59
default logger already configured


In [3]:
# %%
from DeepRL.utils import run_episodes
agent.task._plot = agent.task._plot2 = None
try:    
    run_episodes(agent)
except KeyboardInterrupt as e:
    save_ddpg(agent)
    raise(e)

[0.00111786 0.00253082 0.00105869 ... 0.00103716 0.0032753  0.00096329]
Total Steps: 0  Average fitness: -0.1267209468715485
[0.00100731 0.00191549 0.00175459 ... 0.00096243 0.00094733 0.00119723]
[0.00099538 0.00113958 0.00135647 ... 0.00108749 0.00094196 0.00107708]
[0.0012604  0.00103086 0.00143123 ... 0.00096727 0.00090631 0.00103452]
[0.00079821 0.00101684 0.00104737 ... 0.00084247 0.00106509 0.00083192]
[0.00103349 0.00104964 0.00117844 ... 0.00080456 0.00078569 0.00076817]
[0.00078659 0.00081467 0.0013664  ... 0.00077102 0.00081621 0.00083827]
[0.00091122 0.00134275 0.00148334 ... 0.00081749 0.00070919 0.00096184]
[0.00081688 0.00106123 0.00105184 ... 0.00070458 0.00073786 0.000739  ]
[0.0007016  0.00118795 0.00087545 ... 0.00082007 0.0013577  0.00078953]
[0.00075016 0.00085139 0.00086996 ... 0.00062842 0.00084309 0.00074964]
Total Steps: 10  Average fitness: 2.679048800832978
[0.00070702 0.0006758  0.0005528  ... 0.00065988 0.00077405 0.00069281]
[0.00071975 0.00058656 0.000607

[2.19070203e-05 1.95640022e-05 2.28485568e-05 ... 2.14780690e-05
 3.72041358e-05 2.03034506e-05]
[1.95874080e-05 2.78997423e-05 2.39067149e-05 ... 2.08987822e-05
 2.07794014e-05 2.98717553e-05]
[2.28332609e-05 2.37448352e-05 1.87146515e-05 ... 2.72020792e-05
 1.80190586e-05 2.02666447e-05]
[1.79608762e-05 1.83782949e-05 1.84764269e-05 ... 2.51594075e-05
 1.74406426e-05 2.40847682e-05]
[1.91990040e-05 2.44101075e-05 1.74467531e-05 ... 2.38189577e-05
 2.09265877e-05 2.29193033e-05]
[1.99258065e-05 2.68056843e-05 2.31176742e-05 ... 2.31181880e-05
 1.86134045e-05 2.15211001e-05]
[1.91341689e-05 1.85580088e-05 1.65580928e-05 ... 3.44742871e-05
 1.70771690e-05 1.67779517e-05]
[1.99471521e-05 2.57891839e-05 1.86914772e-05 ... 1.83387477e-05
 1.67520609e-05 1.76950933e-05]
[1.81234596e-05 2.52679899e-05 1.65610654e-05 ... 1.56614255e-05
 1.62022963e-05 1.79689845e-05]
Total Steps: 100  Average fitness: -0.23838164625954011
[1.84213006e-05 1.80334176e-05 1.61182834e-05 ... 1.77045094e-05
 1.730

[1.03829271e-05 1.27337491e-05 1.57175585e-05 ... 1.67619413e-05
 1.05550124e-05 1.08012781e-05]
[1.15977782e-05 1.26633474e-05 1.18827923e-05 ... 1.17210109e-05
 1.12815301e-05 1.21329417e-05]
[1.32067974e-05 1.77761707e-05 1.30916017e-05 ... 1.25341204e-05
 1.11278193e-05 1.34619695e-05]
[1.26740779e-05 1.02900470e-05 1.09256632e-05 ... 1.12586510e-05
 1.36464844e-05 1.03051094e-05]
[1.05755007e-05 1.49838927e-05 1.32603490e-05 ... 1.11749859e-05
 1.13099030e-05 1.22484134e-05]
[1.10516965e-05 1.06458945e-05 1.19680751e-05 ... 1.07882329e-05
 1.05078130e-05 1.12694907e-05]
[1.45450139e-05 1.23957856e-05 1.25909712e-05 ... 1.40551535e-05
 1.16995874e-05 1.28016581e-05]
[1.06239940e-05 1.15767950e-05 1.14919087e-05 ... 1.14017646e-05
 1.29533779e-05 1.08350958e-05]
[1.35534972e-05 1.29944887e-05 1.04040181e-05 ... 1.11409671e-05
 1.22112755e-05 1.13855414e-05]
Total Steps: 180  Average fitness: 0.2544700472153374
[1.24784337e-05 1.12413771e-05 1.54637406e-05 ... 1.09363931e-05
 1.26511

[1.25578313e-05 1.12500822e-05 1.10597908e-05 ... 1.11773592e-05
 1.13434255e-05 1.33589764e-05]
[1.36945743e-05 1.23985710e-05 1.18692247e-05 ... 1.31181206e-05
 1.10474065e-05 1.19404599e-05]
[1.28044887e-05 1.38762923e-05 1.08614023e-05 ... 1.05951654e-05
 1.03655364e-05 1.05386626e-05]
[1.15919169e-05 1.26463857e-05 1.09304326e-05 ... 1.06712853e-05
 1.00204491e-05 1.16349087e-05]
[1.26642280e-05 1.13933661e-05 1.86232178e-05 ... 1.05009091e-05
 1.24908820e-05 1.06685557e-05]
[1.24433829e-05 1.02417695e-05 1.04335493e-05 ... 1.60017357e-05
 1.00663008e-05 1.15928538e-05]
[1.14847897e-05 1.04068468e-05 1.07532613e-05 ... 1.95710604e-05
 1.73606961e-05 1.15168734e-05]
[1.38032643e-05 1.07234942e-05 1.52239191e-05 ... 1.34047785e-05
 1.20806861e-05 1.16786144e-05]
[1.03505883e-05 1.12036148e-05 1.09303651e-05 ... 1.07936363e-05
 1.37532250e-05 1.09275453e-05]
Total Steps: 260  Average fitness: -0.016784095116622422
[1.07525002e-05 1.04263118e-05 1.10145885e-05 ... 1.39923071e-05
 1.05

[1.07697229e-05 1.11295040e-05 1.38633653e-05 ... 1.25270497e-05
 1.28417565e-05 1.61964978e-05]
[1.22761728e-05 1.19905759e-05 1.12639984e-05 ... 1.09486998e-05
 1.17150235e-05 1.19069304e-05]
[1.21902578e-05 1.34658234e-05 1.13387907e-05 ... 1.02210623e-05
 1.18245001e-05 1.15939241e-05]
[1.08422033e-05 1.21675459e-05 1.17750308e-05 ... 1.06476618e-05
 1.40845754e-05 1.47895261e-05]
[1.08812666e-05 1.08530885e-05 1.53809006e-05 ... 1.61501438e-05
 1.16867661e-05 1.52268169e-05]
[1.08304529e-05 1.09344292e-05 1.43253152e-05 ... 1.32620286e-05
 1.11285284e-05 1.10989188e-05]
[1.01993181e-05 1.07274727e-05 1.57862232e-05 ... 1.23753337e-05
 1.08532746e-05 1.12180481e-05]
[1.37594243e-05 1.71896652e-05 1.24371042e-05 ... 1.04617569e-05
 1.28265674e-05 1.39654885e-05]
[1.27947438e-05 1.19827570e-05 1.18301592e-05 ... 1.12177243e-05
 1.65850613e-05 1.22631328e-05]
Total Steps: 340  Average fitness: 1.1771892378891597
[1.16406957e-05 1.10445043e-05 1.19094417e-05 ... 1.06999552e-05
 1.44582

...

INFO:gym:episode 2878, reward -0.043473, avg reward 0.208368, total steps 4816, episode step 1280
[2020-05-01 20:28:07,979] episode 2878, reward -0.043473, avg reward 0.208368, total steps 4816, episode step 1280
INFO:gym:Testing...
[2020-05-01 20:28:07,980] Testing...


[1.43777008e-05 1.08084532e-05 1.27103770e-05 ... 1.11478845e-05
 1.29524171e-05 1.14413045e-05]


INFO:gym:Avg reward 0.299654(0.000000)
[2020-05-01 20:28:09,840] Avg reward 0.299654(0.000000)
INFO:gym:episode 2879, reward -1.351612, avg reward 0.166094, total steps 4818, episode step 1280
[2020-05-01 20:28:11,324] episode 2879, reward -1.351612, avg reward 0.166094, total steps 4818, episode step 1280


[1.28700993e-05 1.23248863e-05 1.36010034e-05 ... 1.16106982e-05
 1.19603705e-05 1.19772921e-05]
data/DDPGAgent-ddpg-20200501_22-22-59-model-PortfolioEnv.bin


KeyboardInterrupt: 

In [4]:
def test_algo(env, algo, seed=0):
    """
    Runs and algo from https://github.com/Marigold/universal-portfolios on env
    
    https://github.com/Marigold/universal-portfolios/commit/e8970a82427522ef11b1c3cbf681e18b5fe8169c
    """
    env.seed(0)
    np.random.seed(0)

    state = env.reset()
    for i in range(env.unwrapped.sim.steps):
        
        history= pd.DataFrame(state[0,:,:], columns=env.unwrapped.src.asset_names)
        # MPT wants a cash column, and it should be first
        history['CASH']=1
        history=history[['CASH'] + env.unwrapped.src.asset_names]
#         cols = list(history.columns)
#         cols[0]='CASH'
#         history.columns = cols
        
        x=history.iloc[-1]
        
        last_b = env.unwrapped.sim.w0#[1:]

        algo.init_step(history)
        # some don't want history
        try:
            action = algo.step(x, last_b, history)
        except TypeError:
            action = algo.step(x, last_b)
        
        # might by dataframe
        action = getattr(action, 'value', action)
        
        # For upt
        if isinstance(action, np.matrixlib.defmatrix.matrix):
            action = np.array(action.tolist()).T[0]
            
        

        state, reward, done, info = env.step(action)

        if done:
            break   
    df = pd.DataFrame(env.unwrapped.infos)
    df.index = pd.to_datetime(df['date']*1e9)
    return df['portfolio_value'], df


In [5]:
df_test = pd.read_hdf('./data/poloniex_30m.hf',key='test')
test_steps=500 #5000
env_test = task_fn_test()
agent.task = env_test
agent.config.max_episode_length = test_steps
agent.task.reset()
np.random.seed(0)

# run in deterministic mode, no training, no exploration
agent.episode(True)
# agent.task.render('notebook')
# agent.task.render('notebook', True)

# df = pd.DataFrame(agent.task.unwrapped.infos)
# df.index = pd.to_datetime(df['date']*1e9)


# %%
df = pd.read_csv('ddpg_cem_test_results.csv')
df

# %%
len(df_test)


8291

In [6]:
from UniversalPortfolios.universal import algos
env = task.unwrapped
price_cols = [col for col in df.columns if col.startswith('price')]
for col in price_cols:
    df[col]=df[col].cumprod()

df = df[price_cols + ['portfolio_value']]
    
algo_dict=dict(
    # Pick the same is in https://arxiv.org/pdf/1706.10059.pdf
    # Benchmarks
#     UCRP=algos.UP(),
    
    # Follow the winner
    BestSoFar=algos.BestSoFar(cov_window=env_test.unwrapped.src.window_length-1),
#     UniversalPortfolio=algos.UP(eval_points=1000),
    ONS=algos.ONS(),
    
    # Follow the loser
#     OnlineMovingAverageReversion=algos.OLMAR(window=env.src.window_length-1, eps=10), 
    RMR=algos.RMR(window=env_test.unwrapped.src.window_length-1, eps=10),
#     PassiveAggressiveMeanReversion=algos.PAMR(),
    
    # Pattern matching
    #     CorrelationDrivenNonparametricLearning=algos.CORN(window=30),
)
for name, algo in algo_dict.items():
    print(name)
    perf, _ = test_algo(env_test, algo)
    perf.index=df.index
    df[name]=perf

# put portfolio value at end so we plot it on top and can therefore see it
cols = list(df.columns.drop('portfolio_value'))+['portfolio_value']
df=df[cols]


df.plot(alpha=0.5)


KeyError: "None of [Index(['portfolio_value'], dtype='object')] are in the [columns]"